In [8]:
#WEB SCRAPPING:

import os
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

#Extract URL - Scrapper:

url = 'https://en.wikipedia.org/wiki/List_of_countries_and_dependencies_by_population'
html = requests.get(url).content
soup = BeautifulSoup(html, "lxml")

table = soup.find_all('table',{'class':'wikitable sortable mw-datatable'})[0]
table

# tr represent the table rows:
rows = table.find_all('tr')
rows_parsed = [row.text for row in rows]
rows_parsed

#Parsering:

def smart_parser(row_text):
    row_text = row_text.replace('\nPopulation\n\n\n', '\nPopulation\n').strip('\n')
    row_text = row_text.replace('\n\n', '\n').strip('\n')
    row_text = row_text.replace('\nPopulation\n% of World\n', '\nPopulation % of World\n').strip('\n')
    row_text = re.sub('\[\d\]', '', row_text)
    return list(map(lambda x: x.strip(), row_text.split('\n')))

well_parsed = list(map(lambda x: smart_parser(x), rows_parsed))

colnames = well_parsed[0]
data = well_parsed[1:]

#Creating DF:
df_web_scrapping = pd.DataFrame(data, columns=colnames)

df_web_scrapping['Country(or dependent territory)']=df_web_scrapping['Country(or dependent territory)'].replace(['\[(.*?)\]','\(([^\)]+)\)'], ['',''], regex=True)

df_web_scrapping.columns = ['Rank', 'country','Population',' % of World Population','Date','Source']

#Exporting to CSV: 

df_web_scrapping.to_csv('/Users/MIGUEL/Desktop/CLAB1/Ironhack-Module-1-Project---Pipelines-/data/processed/df_web_scrapping.csv', index=False)
df_web_scrapping

,Rank,country,Population,% of World Population,Date,Source
0,1,China,"1,400,795,960",18.1%,8 Jan 2020,National population clock
1,2,India,"1,357,082,950",17.5%,8 Jan 2020,National population clock
2,3,United States,"330,552,139",4.26%,8 Jan 2020,National population clock
3,4,Indonesia,"266,911,900",3.44%,1 Jul 2019,National annual projection
4,5,Pakistan,"218,209,000",2.81%,8 Jan 2020,National population clock
...,...,...,...,...,...,...
236,–,Tokelau,"1,400",0.0000180%,1 Jul 2018,National annual estimate[94]
237,–,Vatican City,799,0.0000103%,1 Jul 2019,UN projection
238,–,Cocos Islands,538,0.00000693%,30 Jun 2018,National estimate[197]
239,–,Pitcairn Islands,50,0.000000644%,1 Jan 2019,National estimate[198]
